# Pretrained Model, classification, Elastic Net

In [18]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'greater_zero')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [4]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,0.565417,0.438914,0.512821,0.490598,0.590909,0.54931,0.514793,0.542735,0.903114,0.923351,0.951961,0.671123,0.806561,0.826923,0.808007,0.964187,0.955556,0.913223,0.657343,0.783217,0.843434,0.962222,0.759091,0.714103,0.903846,0.875,0.822314,0.512238,0.694056,0.727273,0.718935,0.652367,0.662393,0.89645,0.931624,0.91821
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.607495,0.460784,0.515929,0.500855,0.536131,0.434911,0.519724,0.59188,0.880623,0.901961,0.82451,0.522727,0.719457,0.297511,0.55719,0.96786,0.934343,0.884298,0.722611,0.470862,0.645623,0.864444,0.721212,0.507692,0.5,0.537963,0.900826,0.736014,0.624126,0.585859,0.85503,0.387574,0.657051,0.718935,0.647436,0.802469
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.628534,0.503771,0.54157,0.578632,0.558275,0.542406,0.610454,0.541311,0.799308,0.672906,0.647059,0.46123,0.649321,0.496606,0.598039,0.947658,0.835354,0.823691,0.600233,0.543124,0.479798,0.848889,0.786364,0.585897,0.710256,0.450926,0.838843,0.501748,0.601399,0.464646,0.745562,0.618343,0.632479,0.689349,0.599359,0.712963
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,0.78501,0.536199,0.302253,0.347009,0.576923,0.548323,0.588757,0.666667,0.930796,0.523173,0.34902,0.589572,0.593891,0.331448,0.601307,0.719927,0.629293,0.698347,0.452214,0.47669,0.380471,0.675556,0.543939,0.382051,0.642308,0.441667,0.72314,0.417832,0.536713,0.487374,0.653846,0.461538,0.57265,0.727811,0.642094,0.743827
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.682446,0.585219,0.686869,0.624786,0.648019,0.523669,0.623274,0.678775,0.745675,0.468806,0.657843,0.59492,0.536199,0.744344,0.762255,0.831956,0.729293,0.753444,0.460373,0.655012,0.679293,0.837778,0.734848,0.484615,0.902564,0.750926,0.760331,0.43007,0.746503,0.703283,0.659763,0.491124,0.667735,0.91716,0.885684,0.950617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,0.661407,0.535445,0.515929,0.384615,0.624709,0.552268,0.500986,0.628205,0.854671,0.787879,0.766667,0.716578,0.371041,0.669683,0.611928,0.758494,0.743434,0.703857,0.424242,0.652681,0.531145,0.773333,0.584848,0.384615,0.707692,0.476852,0.599174,0.494755,0.58042,0.578283,0.825444,0.531065,0.577991,0.695266,0.668803,0.694444
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,0.802104,0.263198,0.637141,0.400855,0.695804,0.572978,0.585799,0.539886,0.83564,0.361854,0.377451,0.163102,0.542986,0.345023,0.449346,0.775023,0.522222,0.782369,0.456876,0.463869,0.51431,0.773333,0.486364,0.485897,0.589744,0.383333,0.855372,0.363636,0.562937,0.609848,0.781065,0.511834,0.606838,0.739645,0.577991,0.694444
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.690335,0.427602,0.434343,0.455556,0.545455,0.639053,0.543393,0.58547,0.780277,0.631907,0.609804,0.469251,0.49095,0.464932,0.522059,0.752984,0.675758,0.636364,0.498834,0.441725,0.41835,0.762222,0.540909,0.478205,0.64359,0.512963,0.710744,0.51049,0.43007,0.435606,0.674556,0.454142,0.457265,0.636095,0.639957,0.625
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.741617,0.365008,0.717949,0.347863,0.722611,0.731755,0.522682,0.670228,0.678201,0.328877,0.603922,0.390374,0.354072,0.552036,0.409314,0.947658,0.617172,0.913223,0.688811,0.390443,0.302189,0.726667,0.589394,0.369231,0.520513,0.240741,0.859504,0.702797,0.512238,0.395202,0.840237,0.560651,0.719017,0.633136,0.588675,0.759259


In [5]:
classification = True
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "MEM_score", "Apoe", "IDs"]
target = "prmdiag"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [6]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
30,0.0,1.774171,-1.149848,-0.727818,0.223612,0.510248,0.149042,-0.046593,0.286511,0.995915,-2.293606,-1.497364,1.393730,0.965808,0.504752,0.810106,2.245960,-0.321284,0.339522,-1.136481,-0.394558,-1.078407,0.780567,-0.317430,0.899392,-1.096511,-0.388152,0.497019,0.002149,0.069702,-0.251668,0.193886,-0.150577,0.318213,2.098086,-0.286719,0.466680,0.283776,-1.602942,-1.112702
792,1.0,1.106080,-1.149848,2.001953,2.064882,-1.614341,1.116766,-2.313764,2.250677,1.332024,-1.696687,-0.611557,-0.597619,-0.107328,0.634571,-0.966817,-0.020573,-1.139360,-1.354596,0.595144,-0.655207,0.296930,1.889529,-1.717650,-0.933406,-0.431717,-2.078170,-0.237103,-2.047002,-1.237929,-0.043123,1.607669,-1.730249,-0.182639,-0.014712,-2.706186,-2.217331,0.772852,1.239828,-0.170717
497,0.0,0.103945,0.469414,-0.386597,-1.578894,-0.481696,-1.104599,-0.383858,-1.433927,0.284916,-0.046381,-0.437869,1.753868,1.604187,0.602116,-0.917730,-1.049870,-1.312065,-0.283855,1.055449,0.414351,-1.150039,-0.552053,-1.150690,0.417781,1.475954,-0.267436,-0.494690,0.449499,0.806395,0.634647,0.875828,0.082784,-0.305070,-0.283613,0.272886,0.310213,-0.246055,-0.110163,-0.478304
594,1.0,1.440126,-1.149848,1.660732,0.242994,-0.784204,-1.141255,-0.889756,-0.200946,-0.335593,0.012140,0.387148,1.054777,1.411573,1.510847,1.752562,0.998327,1.651190,1.454266,-0.193951,0.405363,0.755376,0.780567,2.044904,1.621809,-0.113771,0.698288,0.780365,0.983433,2.519208,-0.095259,1.541138,1.375243,1.219747,0.062117,1.062916,0.779614,0.813608,1.538384,1.713254
712,0.0,0.772035,0.469414,-0.727818,-0.512896,0.418792,1.497990,1.096361,0.587587,0.478825,0.550538,0.856105,0.652270,-1.279524,-1.117981,-0.574126,1.247854,1.533024,1.432264,1.033530,0.998563,1.715247,0.752610,1.615389,0.297378,1.649379,1.452761,0.715968,1.286476,-0.998504,1.103873,0.027559,1.034177,0.162392,0.407848,0.750196,0.743506,0.446802,1.058098,1.636357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0.0,-0.898191,-1.149848,0.978289,0.010413,1.157474,0.515604,0.852780,0.903001,0.052225,0.023844,0.317673,1.965713,2.027938,0.504752,1.291151,0.759198,-0.175848,1.146244,0.792418,1.474921,0.425868,-0.123378,0.679045,1.909438,-0.113771,1.241509,-0.726518,0.651528,2.150861,0.269694,-0.621118,-0.114675,0.752285,0.599920,-0.517144,-1.206313,0.079995,1.291751,0.867389
332,1.0,0.605013,0.469414,-0.386597,0.708157,1.347420,0.867504,-0.730492,-0.401664,1.810333,-0.140016,0.005035,1.414915,0.442998,-0.825889,-0.063629,0.811183,-0.357643,0.926229,-0.961126,0.099775,-0.089883,0.743291,0.902393,1.514784,0.088557,0.487036,-0.919708,0.290762,0.493301,-1.346528,-0.055605,-0.276233,-0.060209,0.676749,-0.138588,0.225962,-2.080088,-1.031792,-1.939343
228,1.0,0.103945,0.469414,2.001953,-2.315402,0.228845,0.310330,0.028355,-0.645393,-0.400230,-1.134881,-0.958932,-3.330427,-0.487054,-0.955708,0.348695,-1.081061,-0.375822,-0.826559,-1.421432,-1.185492,-0.691593,0.733972,0.498649,0.437848,-2.657332,-0.825746,0.084880,-0.935842,0.751143,-1.972162,1.092054,-0.742954,0.329343,-2.204338,-0.237342,-0.917451,-2.406138,-1.252464,-2.842880
397,1.0,2.108216,-1.149848,-0.727818,-0.008969,-0.946010,-0.891993,-1.395654,-0.416001,-0.516575,-1.146585,-1.671052,-0.385773,-0.057799,0.951004,-0.525040,-1.351381,0.024126,-0.921899,-0.303547,-0.070995,-0.662940,-0.216568,-0.996064,-0.933406,0.348694,-1.610397,-0.675001,-1.599652,-0.648574,-0.981575,-0.521321,-1.263528,-1.028523,-1.935437,-0.533603,-0.351763,-0.449837,-0.252951,-0.420631


# Train Model
Evaluate AUC and accuracy on test data

In [7]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, 
                          classification = True, 
                          n_alphas_logreg = 10, 
                          l1_ratios_logreg= np.linspace(0,  1, 11).tolist(), 
                          cv_logreg = 5,
                          max_iter = 500) 

In [9]:
predictions = model.predict(Xtest)
accuracy_score(ytest, predictions)

0.6808510638297872

In [10]:
pred_prob = model.predict_proba(Xtest)
auc = roc_auc_score(ytest, pred_prob[:, 1])
auc

0.7239010989010989

In [11]:
model.l1_ratio_

array([1.])

In [12]:
model.C_

array([0.35938137])

# Save Model

In [13]:
import pickle

In [14]:
filename ='pretrained_elnet_class_agg_zero.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [19]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_class_agg_zero.sav", 'rb'))